In [ ]:
import tensorflow 
import numpy as np
import os
import pandas as pd

In [ ]:
csv_file = 'Nat_Gas.csv'

In [ ]:
df = pd.read_csv(csv_file)
df

In [ ]:
df.index = pd.to_datetime(df['Dates'], format='%m/%d/%y')

In [ ]:
df

In [ ]:
temp = df['Prices']
temp.plot()

In [ ]:
def df_to_xy(df, window_size=12):
    dfp = df.to_numpy()
    X = []
    y = []
    for i in range(len(dfp)-window_size):
        row = [[a] for a in dfp[i:i+window_size]]
        X.append(row)
        label = dfp[i+window_size]
        y.append(label)
    return np.array(X), np.array(y)


In [ ]:
windows = 12
X, y = df_to_xy(temp, windows)
X.shape, y.shape

In [ ]:
X_train, y_train = X[:40], y[:40]
X_val, y_val = X[40:45], y[40:45]
X_test, y_test = X[45:], y[45:]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(InputLayer((12,1)))
model.add(LSTM(20))
model.add(Dense(10, "relu"))
model.add(Dense(1, "linear"))
model.summary()


In [ ]:
os.makedirs("model", exist_ok=True)
cp = ModelCheckpoint("model/best_model.keras", save_best_only=True)
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=.001), metrics=[RootMeanSquaredError()])

In [ ]:
model.fit(X_train, y_train, epochs=1100, validation_data=(X_val, y_val), callbacks=[cp])

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("model/best_model.keras")

In [ ]:
train_predictions = model.predict(X_train).flatten()
train_results = pd.DataFrame(data={"Train Predictions": train_predictions, "Actuals": y_train})
train_results.plot()